In [2]:
!pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


In [5]:
import random
from deap import base, creator, tools, algorithms

# Definición del problema como maximización
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Parámetros
population_size = 100
generations = 3
mutation_rate = 0.2
crossover_rate = 0.5

# Función objetivo
def fitness_function(individual):
    x = individual[0]
    return (x**(2*x) - 1,)

# Crear herramientas
toolbox = base.Toolbox()
toolbox.register("x", random.uniform, 0, 10)  # Rango de valores para x
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.x, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=mutation_rate)
toolbox.register("select", tools.selTournament, tournsize=3)

# Algoritmo genético
population = toolbox.population(n=population_size)

for generation in range(generations):
    # Evaluar la población
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Selección
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # Cruzamiento
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < crossover_rate:
            toolbox.mate(child1, child2)
            del child1.fitness.values  # Marcar como no evaluado
            del child2.fitness.values  # Marcar como no evaluado

    # Mutación
    for mutant in offspring:
        if random.random() < mutation_rate:
            toolbox.mutate(mutant)
            del mutant.fitness.values  # Marcar como no evaluado

    # Reemplazo
    population[:] = offspring

# Evaluar la población final
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

# Obtener el mejor individuo
fits = [ind.fitness.values[0] for ind in population]
best_idx = fits.index(max(fits))
best_ind = population[best_idx]

print(f"Mejor individuo: {best_ind}, f(x) = {best_ind[0]**(2*best_ind[0]) - 1}")


Mejor individuo: [12.03238864382394], f(x) = 9.963838716870826e+25


In [6]:
import random

# Parámetros
population_size = 100
generations = 3
mutation_rate = 0.2
crossover_rate = 0.5
x_range = (0, 10)

# Función objetivo
def fitness_function(x):
    return x**(2*x) - 1

# Crear población inicial
population = [random.uniform(*x_range) for _ in range(population_size)]

# Algoritmo genético
for generation in range(generations):
    # Evaluar la población
    fitnesses = [fitness_function(x) for x in population]

    # Selección (torneo)
    selected = []
    for _ in range(population_size):
        competitors = random.sample(population, 3)
        selected.append(max(competitors, key=fitness_function))

    # Cruzamiento
    offspring = []
    for i in range(0, population_size, 2):
        parent1 = selected[i]
        parent2 = selected[i + 1] if i + 1 < population_size else selected[0]
        if random.random() < crossover_rate:
            # Cruzamiento uniforme
            child = (parent1 + parent2) / 2  # Cruzamiento simple
        else:
            child = parent1
        offspring.append(child)

    # Mutación
    for i in range(len(offspring)):
        if random.random() < mutation_rate:
            offspring[i] += random.gauss(0, 1)  # Mutación gaussiana

    # Reemplazo
    population = offspring

# Obtener el mejor individuo
best_ind = max(population, key=fitness_function)

print(f"Mejor individuo: {best_ind}, f(x) = {fitness_function(best_ind)}")


Mejor individuo: 11.257517910064506, f(x) = 4.7126897561424366e+23
